In [1]:
"""! pip install pytorch_pretrained_bert
#! pip install pytorch_transformers
! pip install seqeval
! pip install transformers"""

'! pip install pytorch_pretrained_bert\n#! pip install pytorch_transformers\n! pip install seqeval\n! pip install transformers'

In [1]:
import os
import json
import pandas as pd
import numpy as np
from transformers import BertModel,BertTokenizer, BertForTokenClassification
from tensorflow.keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch
from tqdm import tqdm, trange


I0505 14:50:35.425671 47764350328512 file_utils.py:41] PyTorch version 1.2.0 available.
I0505 14:50:42.227373 47764350328512 file_utils.py:57] TensorFlow version 2.1.0 available.


In [2]:
from torch.optim import Adam
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [3]:
# from pytorch_pretrained_bert import WEIGHTS_NAME, CONFIG_NAME, BertForQuestionAnswering

In [4]:
"""from google.colab import drive
drive.mount("/content/drive/")"""

'from google.colab import drive\ndrive.mount("/content/drive/")'

In [ ]:
"""%cd drive/My\ Drive/data
!ls"""

'%cd drive/My\\ Drive/data\n!ls'

In [ ]:
output_dir = "./biobert_v1.1_pubmed/"
tokenizer = BertTokenizer.from_pretrained(output_dir, do_lower_case=True)  # Add specific options if needed

I0505 14:50:43.679851 47764350328512 tokenization_utils.py:420] Model name './biobert_v1.1_pubmed/' not found in model shortcut name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese, bert-base-german-cased, bert-large-uncased-whole-word-masking, bert-large-cased-whole-word-masking, bert-large-uncased-whole-word-masking-finetuned-squad, bert-large-cased-whole-word-masking-finetuned-squad, bert-base-cased-finetuned-mrpc, bert-base-german-dbmdz-cased, bert-base-german-dbmdz-uncased, bert-base-finnish-cased-v1, bert-base-finnish-uncased-v1, bert-base-dutch-cased). Assuming './biobert_v1.1_pubmed/' is a path, a model identifier, or url to a directory containing tokenizer files.
I0505 14:50:43.682863 47764350328512 tokenization_utils.py:449] Didn't find file ./biobert_v1.1_pubmed/added_tokens.json. We won't load it.
I0505 14:50:43.683643 47764350328512 tokenization_utils.py:449] Did

#### Load model

In [ ]:
!nvidia-smi
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

torch.cuda.get_device_name(0) 

device

Tue May  5 14:50:44 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:18:00.0 Off |                    0 |
| N/A   38C    P0    58W / 300W |      0MiB / 16160MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

device(type='cuda')

#### Load Data from Test set

In [ ]:
#from my_process_json import *
from create_processed_data import *

In [ ]:
train_data_processor = DataProcessor('train',size=None,tokenizer=tokenizer)
valid_data_processor =  DataProcessor('dev',size=None,tokenizer=tokenizer)

In [ ]:
tags_vals = train_data_processor.get_tags_vals()
tr_inputs,tr_masks,tr_tags,tr_ttids,_ = train_data_processor.get_processed_data()

val_inputs,val_masks,val_tags,val_ttids, _ = valid_data_processor.get_processed_data()


In [ ]:
train_data_processor.dataset_size

In [ ]:
val_tags

In [ ]:
torch.cuda.empty_cache() 

In [ ]:
tr_inputs = torch.tensor(tr_inputs)
tr_masks = torch.tensor(tr_masks)
tr_tags = torch.tensor(tr_tags)
tr_ttids = torch.tensor(tr_ttids)

val_inputs = torch.tensor(val_inputs)
val_masks = torch.tensor(val_masks)
val_tags = torch.tensor(val_tags)
val_ttids = torch.tensor(val_ttids)

In [ ]:
bs = 16
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags , tr_ttids)
train_sampler = SequentialSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)

In [ ]:
valid_data = TensorDataset(val_inputs, val_masks, val_tags, val_ttids)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=bs)

In [ ]:
!nvidia-smi

In [ ]:

# Step 2: Re-load the saved model and vocabulary

# Example for a Bert model
model = BertForTokenClassification.from_pretrained(output_dir, num_labels=len(tags_vals))

In [ ]:
model.cuda();

In [ ]:
!nvidia-smi

In [ ]:
FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters()) 
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
optimizer = Adam(optimizer_grouped_parameters, lr=3e-5)

In [ ]:
from seqeval.metrics import f1_score
from seqeval.metrics import classification_report
from torch.nn import CrossEntropyLoss


def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=2).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
print('Initiating training on train set')
closs = CrossEntropyLoss(ignore_index=-100)
epochs = 4
max_grad_norm = 1.0
F_scores = np.zeros(epochs,float)
patience = 6
for e in trange(epochs):
    # TRAIN loopmodel.train()
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    predictions , true_labels = [], []
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels, b_ttids = batch

        logits = model(b_input_ids, token_type_ids=b_ttids,attention_mask=b_input_mask)[0]
        blogits = logits.view(-1, len(tags_vals))
        labels = b_labels.view(-1)
        loss = closs(blogits,labels)

        logits = logits.detach().cpu().numpy()
        predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
        label_ids = b_labels.to('cpu').numpy()
        true_labels.append(label_ids)
        
        loss.backward()
        # track train loss
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
        
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        # update parameters
        optimizer.step()
        optimizer.zero_grad()
        
    # print train loss per epoch
    print("Train loss: {}".format(tr_loss/nb_tr_steps))
    pred_tags = [tags_vals[p_i] for p in predictions for p_i in p]
    valid_tags = [tags_vals[l_ii] if l_ii!=-100 else '[PAD]' for l in true_labels for l_i in l for l_ii in l_i]
    c_f1 = f1_score(valid_tags, pred_tags)
    rep = classification_report(valid_tags, pred_tags)
    print('Training classification report:',rep)
    print("Training F1-Score: {}".format(c_f1))


    
    # VALIDATION on validation set
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    predictions , true_labels = [], []
    for batch in valid_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels, b_ttids = batch

        with torch.no_grad():
            #tmp_eval_loss = model(b_input_ids, token_type_ids=b_ttids,attention_mask=b_input_mask, labels=b_labels)
            logits = model(b_input_ids, token_type_ids=b_ttids,attention_mask=b_input_mask)[0]
            closs = CrossEntropyLoss(ignore_index=-100)
            blogits = logits.view(-1, len(tags_vals))
            labels = b_labels.view(-1)
            tmp_eval_loss = closs(blogits,labels)


        logits = logits.detach().cpu().numpy()
        predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
        label_ids = b_labels.to('cpu').numpy()
        true_labels.append(label_ids)
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)

        eval_loss += tmp_eval_loss.mean().item()
        eval_accuracy += tmp_eval_accuracy

        nb_eval_examples += b_input_ids.size(0)
        nb_eval_steps += 1

    pred_tags = [[tags_vals[p_i] for p_i in p] for p in predictions]
    valid_tags = [[tags_vals[l_ii] if l_ii!=-100 else '[PAD]' for l_ii in l_i] for l in true_labels for l_i in l]
    print("Validation loss: {}".format(eval_loss/nb_eval_steps))
    print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
    print("Validation F1-Score: {}".format(f1_score(valid_tags, pred_tags)))
    rep = classification_report(valid_tags, pred_tags)
    print(rep)

In [26]:
print('Initiating validation on dev set')
# VALIDATION on validation set
model.eval()
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
predictions , true_labels = [], []
for batch in valid_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels, b_ttids = batch

    with torch.no_grad():
        #tmp_eval_loss = model(b_input_ids, token_type_ids=b_ttids,attention_mask=b_input_mask, labels=b_labels)
        logits = model(b_input_ids, token_type_ids=b_ttids,attention_mask=b_input_mask)[0]
        closs = CrossEntropyLoss(ignore_index=-100)
        blogits = logits.view(-1, len(tags_vals))
        labels = b_labels.view(-1)
        tmp_eval_loss = closs(blogits,labels)


    logits = logits.detach().cpu().numpy()
    predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
    label_ids = b_labels.to('cpu').numpy()
    true_labels.append(label_ids)
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)

    eval_loss += tmp_eval_loss.mean().item()
    eval_accuracy += tmp_eval_accuracy

    nb_eval_examples += b_input_ids.size(0)
    nb_eval_steps += 1

pred_tags = [[tags_vals[p_i] for p_i in p] for p in predictions]
valid_tags = [[tags_vals[l_ii] if l_ii!=-100 else '[PAD]' for l_ii in l_i] for l in true_labels for l_i in l]
print("Validation loss: {}".format(eval_loss/nb_eval_steps))
print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
print("Validation F1-Score: {}".format(f1_score(valid_tags, pred_tags)))
rep = classification_report(valid_tags, pred_tags)
print(rep)

Initiating validation on dev set
Validation loss: 0.05722865881327313
Validation Accuracy: 0.571827906976744
Validation F1-Score: 0.18935939602332186
           precision    recall  f1-score   support

      ans       0.41      0.32      0.36     63389
    [PAD]       0.00      0.00      0.00    101103

micro avg       0.41      0.12      0.19    164492
macro avg       0.16      0.12      0.14    164492



In [24]:
print('Saving model')
from pytorch_pretrained_bert import WEIGHTS_NAME, CONFIG_NAME

output_dir = "./model_notsure/"

# Step 1: Save a model, configuration and vocabulary that you have fine-tuned

# If we have a distributed model, save only the encapsulated model
# (it was wrapped in PyTorch DistributedDataParallel or DataParallel)
model_to_save = model.module if hasattr(model, 'module') else model

# If we save using the predefined names, we can load using `from_pretrained`
output_model_file = os.path.join(output_dir, WEIGHTS_NAME)
output_config_file = os.path.join(output_dir, CONFIG_NAME)

torch.save(model_to_save.state_dict(), output_model_file)
model_to_save.config.to_json_file(output_config_file)
tokenizer.save_vocabulary(output_dir)

Saving model


I0507 10:23:04.902904 47764350328512 modeling.py:230] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


NameError: name 'train_tokenizer' is not defined

('./model_notsure/vocab.txt',)

In [27]:
print('Loading test data')
test_data_processor = DataProcessor('test',size=None,tokenizer=tokenizer)

Loading test data


In [30]:
test_inputs,test_masks,test_tags,test_ttids, _ = test_data_processor.get_processed_data()
test_inputs = torch.tensor(test_inputs)
test_masks = torch.tensor(test_masks)
test_tags = torch.tensor(test_tags)
test_ttids = torch.tensor(test_ttids)
test_data = TensorDataset(test_inputs, test_masks, test_tags, test_ttids)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=bs)

In [ ]:
!nvidia-smi

Thu May  7 12:12:02 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:18:00.0 Off |                    0 |
| N/A   40C    P0    59W / 300W |  15676MiB / 16160MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [ ]:
print('Initiating testing on test set')
# VALIDATION on validation set
model.eval()
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
predictions , true_labels = [], []
for batch in test_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels, b_ttids = batch

    with torch.no_grad():
        #tmp_eval_loss = model(b_input_ids, token_type_ids=b_ttids,attention_mask=b_input_mask, labels=b_labels)
        logits = model(b_input_ids, token_type_ids=b_ttids,attention_mask=b_input_mask)[0]
        closs = CrossEntropyLoss(ignore_index=-100)
        blogits = logits.view(-1, len(tags_vals))
        labels = b_labels.view(-1)
        tmp_eval_loss = closs(blogits,labels)


    logits = logits.detach().cpu().numpy()
    predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
    label_ids = b_labels.to('cpu').numpy()
    true_labels.append(label_ids)
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)

    eval_loss += tmp_eval_loss.mean().item()
    eval_accuracy += tmp_eval_accuracy

    nb_eval_examples += b_input_ids.size(0)
    nb_eval_steps += 1

pred_tags = [[tags_vals[p_i] for p_i in p] for p in predictions]
valid_tags = [[tags_vals[l_ii] if l_ii!=-100 else '[PAD]' for l_ii in l_i] for l in true_labels for l_i in l]
print("Validation loss: {}".format(eval_loss/nb_eval_steps))
print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
print("Validation F1-Score: {}".format(f1_score(valid_tags, pred_tags)))
rep = classification_report(valid_tags, pred_tags)
print(rep)

Initiating validation on dev set
